# 学認連携の設定を行う -- IdP-proxyを利用する

---

学認と連携する設定を行います。CoursewareHubを学認に直接登録せずに、IdP-proxyを利用して学認と連携して認証する設定を行います。

## 概要

このNotebookで設定するCoursewareHubと学認フェデレーションの構成要素を以下に示します。

![構成](images/cw-221-01.png)

このNotebookではCoursewareHubをIdP-proxyを利用して学認連携を設定する手順について記しています。

各コンテナの役割や処理の詳細については https://github.com/NII-cloud-operation/CoursewareHub-LC_jupyterhub-deploy を参照してください。

### 前提条件

このNotebookは構築済みのCoursewareHubに対して学認連携による認証を追加する設定を行います。そのためCoursewareHubが構築済みであることを前提としています。

学認との連携にはIdP-proxyを利用します。IdP-proxyは事前に構築されていることを前提としています。まだIdP-proxyを構築していない場合は「521-IdP-proxyのセットアップ.ipynb」などのNotebookを用いて先にIdP-proxyの構築を行ってください。またCoursewareHubのauth-proxyコンテナとIdP-proxyとの間でメタデータの取得を行います。お互いにアクセス可能となるようファイアウォールやホスト名の名前解決などの設定を事前に行ってください。

### 設定手順

IdP-proxyを利用して学認と連携するための設定手順を以下に示します。

1. mAPグループの作成
1. CoursewareHubに対してIdP-proxyと連携するための設定変更を行う
1. 構築したCoursewareHubのメタデータをIdP-proxyに登録する

このNotebookでの設定手順は学認との連携を設定するためCoursewareHubの設定だけではなく学認のシステムでの設定手順も含まれています。
上に示した手順のうち1は学認のクラウドゲートウェイでの設定となります。手順2がCoursewareHubに対する設定となります。また手順3はIdP-proxyでの設定となります。設定3は別のNotebook「541-IdP-proxyへauth-proxyのメタデータを登録する.ipynb」で実行します。

### グループ名

学認連携を設定する対象となるCoursewareHubのansibleグループ名を指定します。

CoursewareHubを構築したansibleのグループ名を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

グループ名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

### チェック

指定されたグループ名が前提条件を満たしていることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

設定対象となるVCノードにアクセスできることを確認します。

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

グループ名に対応する group_varsファイルが存在していることを確認します。

In [ ]:
!test -f group_vars/{ugroup_name}

CoursewareHubのサービスが実行されていることを確認します。

In [ ]:
cmd = "docker stack services --format '{{.Replicas}}' coursewarehub | grep '1/1' | wc -l"
!ansible {target_hub} -m shell -a "{{% raw %}} [ \$({cmd}) -eq 4 ] {{% endraw %}}"

## mAPグループの作成

[学認クラウドゲートウェイサービス](https://cg.gakunin.jp/)にアクセスしてグループの作成などを行います。手順の詳細については「[GakuNin mAPマニュアル--グループ管理者](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=20873726)」を参照してください。

テストフェデレーションでクラウドゲートウェイの機能（SP検証環境）を利用するには事前に利用申請が必要となります。学認mAPの「[問い合わせ](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=8716731)」などに記されている窓口を通してSP検証環境を利用するための依頼を行ってください。テストフェデレーションSP検証環境のアドレスは利用申請後にメールなどで通知されます。

### グループの作成

学認クラウドゲートウェイのウェブサイトから「グループの作成」を実行してください。

![初期画面](images/cw-211-01.png)

> キャプチャー画面はテストフェデレーションのものです。

グループの作成画面に表示された項目を入力してください。

![グループの作成](images/cw-211-02.png)


CoursewareHubに対して学認との連携を設定する際に、ここで「グループID」として指定した値が必要となります。

### SPコネクタとの接続

作成したグループをサービス(SPコネクタ)に紐づけ、グループに属するユーザがIdP-proxyサービス(SP)を利用できるようにします。

グループとサービス（SPコネクタ）の紐づけは、グループ側からサービスを指定する方法とサービス側（SPコネクタ）からグループを指定して接続する方法があります。ここではグループ管理者がSPコネクタ（サービス）へと接続申請を行う手順を示します。設定方法の詳細については学認クラウドゲートウェイマニュアルの「[サービスを利用する](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=20873789)」を参照してください。また、逆の関連付けとなるSPコネクタからグループ管理者へ接続申請を行う手順については「[SPコネクタにグループを接続する](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=20873907)」を参照してください。

まず、グループの管理者メニューから「利用Webサービス」を選択します。

![グループ画面](images/cw-211-03.png)

「利用するWebサービスを追加」ボタンを押すと、サービスの一覧が表示されるので「521-IdP-proxyのセットアップ.ipynb」で登録したサービスを選択してください。

> SPコネクタとして登録する際に、詳細設定の「Q1 作成するSPコネクタは、検索対象にしますか？」の値を「検索対象としない」と設定した場合はWebサービスの一覧には表示されません。

![サービス一覧画面](images/cw-211-04.png)

CoursewareHubのサービスを選択して申請を行うと承認待ちの状態となります。

![承認待ち画面](images/cw-211-05.png)

SPコネクタの管理者が承認すると申請状況の状態に「承認」と表示されるようになります。

> 自身がSP管理者を兼ねている場合は、SPコネクタの設定画面から承認を行ってください。

![グループ画面](images/cw-211-06.png)

### メンバーの招待

必要に応じてグループの「メンバー招待」メニューからCoursewareHubの利用者に対してグループへの招待を行ってください。

![メンバー招待](images/cw-211-07.png)

## CoursewareHubの設定

CoursewareHubに対して学認連携設定を行います。

### パラメータを指定する

CoursewareHubと連携する学認フェデレーションに関するパラメータを設定します。

#### フェデレーション

参加するフェデレーションがテストフェデレーションであるか否かを次のセルで指定してください。

In [ ]:
# (例)
# enable_test_federation = False   # 運用フェデレーション
# enable_test_federation = True    # テストフェデレーション

enable_test_federation = 

#### IdP-proxy

IdP-proxyのホスト名(FQDN)を指定してください。

In [ ]:
# (例)
# auth_fqdn = 'idp-proxy.example.org'

auth_fqdn = 

####  mAPのグループを指定する

CoursewareHubの利用を許可対象となる利用者を指定します。

指定する値は「[2. mAPグループの作成](#mAPグループの作成)」で作成した学認mAPのグループIDとなります。グループIDの指定はリストの形式で行うので複数のグループIDを指定することができます。

In [ ]:
# (例)
# cg_groups = [
#    'group1-id',
#    'group2-id',
#]

cg_groups = [
    
]

#### パラメータの保存

ここまで指定したパラメータを ansible の変数として `group_vars`ファイルに保存します。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)

cg_group_prefix = (
    "https://cg.gakunin.jp/gr/"
    if not enable_test_federation
    else "https://sptest.cg.gakunin.jp/gr/"
)

for key in ["gakunin_ds_hostname", "cg_fqdn", "metadata_signer_url"]:
    if key in gvars:
        del(gvars[key])

gvars.update({
    'enable_federation': False,
    'enable_test_federation': enable_test_federation,
    'auth_fqdn': auth_fqdn,
    'cg_groups': [
        f'{cg_group_prefix}{x}' if not x.startswith('https://') else x
        for x in cg_groups
    ],
})

with gvars_path.open(mode='w') as f:
    yaml.safe_dump(gvars, stream=f)
    
!cat group_vars/{ugroup_name}

### 証明書の配置

IdP-proxyと連携を行うために、IdP-proxyのサーバ証明書を配置します。出どころなどの情報を必要以上に残さないためにNotebookからの操作ではなく、ターミナルなどから **managerノードに ssh でログインして操作を行ってください**。

manager ノードのIPアドレスを確認します。表示されたIPアドレスに対して、ユーザ名`vcp`と「VCノード作成」のNotebookで設定したSSHの秘密鍵を指定することで manager ノードにsshでログインできます。

In [ ]:
!ansible {target_hub} -c local -m debug -a 'var=servicenet_ip'

IdP-proxyのサーバ証明書を以下のセルの実行結果に表示されるパスに配置してください。

In [ ]:
!ansible {target_hub} -c local -m debug -a 'msg="/home/{{{{ansible_user}}}}/certs/{{{{auth_fqdn}}}}.cer"'

**証明書などの配置を行った後に、これ以降の操作を行ってください。**

証明書が配置されたことを確認します。managerノードに配置されたサーバ証明書の内容を表示してみます。

In [ ]:
!ansible {target_hub} -a \
    'openssl x509 -noout -text -in certs/{{{{auth_fqdn}}}}.cer'

### 設定ファイルの配置、更新

学認連携のために必要となる設定ファイルの配置や更新を行います。

ここで配置、更新するファイルを以下の表に示します。

|パス|用途|
|:---|:---|
|/etc/jupyterhub/hub-const.php|auth-proxyコンテナの設定ファイル(更新)|
|/opt/coursewarehub/docker-compose.yml|docker-composeの設定ファイル(更新)|
|/etc/jupyterhub/simplesamlphp/cert/idp-proxy.cer|メタデータ検証用証明書|


まず実際に設定を変更する前にドライラン（チェックモード）でansibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/setup-gakunin.yml || true

実際に設定変更を行います。

In [ ]:
!ansible-playbook -l {target_hub} playbooks/setup-gakunin.yml

配置したファイルを確認します。

In [ ]:
!ansible {target_hub} -a 'tree -F {{{{jupyterhub_cfg_dir}}}}'

### コンテナの更新

コンテナを起動し直して変更した設定の反映を行います。

起動中のコンテナを一度停止します。

In [ ]:
!ansible {target_hub} -a 'docker stack rm coursewarehub'

コンテナを起動し直します。

In [ ]:
!ansible {target_hub} -a 'chdir={{{{compose_dir}}}} \
    docker stack deploy -c docker-compose.yml coursewarehub'

全てのコンテナが起動するまで待ち合わせを行います。


In [ ]:
import time
import sys


cmd = "docker stack services --format '{{.Replicas}}' coursewarehub | grep '1/1' | wc -l"
for retry in range(18):
    time.sleep(10)
    try:
        !ansible {target_hub} -m shell -a "{{% raw %}} [ \$({cmd}) -eq 4 ] {{% endraw %}}"
        break
    except:
        print('retry', file=sys.stderr)
else:
    !ansible {target_hub} -a 'docker stack services coursewarehub'
    raise RuntimeError("起動処理が完了しませんでした。")

コンテナの起動状態を確認します。

In [ ]:
!ansible {target_hub} -a 'docker stack services coursewarehub'

## メタデータの更新

構築したCoursewareHubのメタデータを IdP-proxy に登録するまでは、学認によるログインを利用できません。IdP-proxyの管理者にCoursewareHubのホスト名(FQDN)を伝えてメタデータの登録を依頼してください。自身でIdP-proxyを管理している場合は「541-IdP-proxyへauth-proxyのメタデータを登録する.ipynb」を実行してください。

メタデータをIdP-proxyに登録せずに学認によるログインを行うと以下のようなエラーとなります。

![メタデータのエラー](images/cw-221-05.png)

## CoursewareHubに学認IdPのアカウントでログインする

学認IdPのアカウントでCoursewareHubにログインできることを確認します。

次のセルを実行すると、CoursewareHubのアドレスを表示します。

In [ ]:
print(f'https://{gvars["master_fqdn"]}')

ログイン画面に表示されている「学認フェデレーションへ」のリンクから学認IdPを利用したログインができます。

![ログイン画面](images/cw-221-02.png)

「学認フェデレーションへ」のリンクから進むと以下のようなIdPの選択画面が表示されます。

> キャプチャー画面はテストフェデレーションのものです。

![IdP選択画面](images/cw-221-03.png)

IdPを選択しログインを行ってください。利用を許可したmAPグループに所属しているユーザであればsingle-userサーバが実行されてJupyter環境を利用することができます。

「学認フェデレーションへ」のリンクが表示されない場合、原因としてCoursewareHubからIdP-proxyのメタデータが取得できないななどの理由が考えられます。次のセルのコメントを外して実行し、エラーとならないことを確認してください。

In [ ]:
# !ansible {target_hub} -a 'curl -sf "https://{{{{auth_fqdn}}}}/simplesaml/module.php/saml/sp/metadata.php/default-sp"'

エラーになる場合はファイアウォールや名前解決の設定を見直してください。設定を変更して上のセルがエラーとならずに実行できるようになったら次のセルのコメントを外して実行してください。auth-proxyコンテナが再起動されメタデータが更新されます。

In [ ]:
# !ansible {target_hub} -a 'docker service update --force coursewarehub_auth-proxy'